In [1]:
hooktheory_midi_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/ProcessedData/midi_from_json_songs"
# model_output_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Evaluation/data_for_testing/model_output/topk15-t1.0-ngram0"
model_output_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/output/topk15-t1.0-ngram0"

In [2]:
import pretty_midi
import numpy as np
from collections import Counter
import os
import plotly.express as px

In [3]:
# current_idx = 0

# indices_to_analyze = [
#     0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
#     # 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
#     # 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
#     # 100, 101, 102, 103, 104, 105, 106, 107, 108, 109
# ]

In [4]:
def load_midi(file_path):
    return pretty_midi.PrettyMIDI(file_path)

def get_notes(midi_data):
    notes = []
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                notes.append(note)
    return sorted(notes, key=lambda note: note.start)

def pitch_counts(notes):
    pitches = [note.pitch for note in notes]
    return dict(Counter(pitches))

def pitch_class_distribution(notes):
    pitch_classes = [note.pitch % 12 for note in notes]
    return dict(Counter(pitch_classes))

def pitch_class_transition_matrix(notes):
    pitch_classes = [note.pitch % 12 for note in notes]
    transition_matrix = np.zeros((12, 12))
    for i in range(len(pitch_classes) - 1):
        transition_matrix[pitch_classes[i], pitch_classes[i + 1]] += 1
    return transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

def pitch_range(notes):
    pitches = [note.pitch for note in notes]
    return max(pitches) - min(pitches)

def average_pitch_intervals(notes):
    pitch_intervals = [notes[i + 1].pitch - notes[i].pitch for i in range(len(notes) - 1)]
    return np.mean(pitch_intervals), np.std(pitch_intervals)

def average_inter_onset_intervals(notes):
    inter_onset_intervals = [notes[i + 1].start - notes[i].start for i in range(len(notes) - 1)]
    return np.mean(inter_onset_intervals), np.std(inter_onset_intervals)

def note_count(notes):
    return len(notes)

def note_length_transition_matrix(notes):
    note_lengths = [note.end - note.start for note in notes]
    unique_lengths = sorted(set(note_lengths))
    length_to_idx = {length: idx for idx, length in enumerate(unique_lengths)}
    transition_matrix = np.zeros((len(unique_lengths), len(unique_lengths)))
    
    for i in range(len(note_lengths) - 1):
        current_length = note_lengths[i]
        next_length = note_lengths[i + 1]
        transition_matrix[length_to_idx[current_length], length_to_idx[next_length]] += 1
    
    return transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

def analyze_midi(file_path):
    midi_data = load_midi(file_path)
    notes = get_notes(midi_data)

    analysis = {}
    
    if len(notes) != 0:
        analysis = {
            'pitch_counts': pitch_counts(notes),
            'pitch_class_distribution': pitch_class_distribution(notes),
            # 'pitch_class_transition_matrix': pitch_class_transition_matrix(notes),
            # 'pitch_range': pitch_range(notes),
            # 'average_pitch_intervals': average_pitch_intervals(notes),
            # 'average_inter_onset_intervals': average_inter_onset_intervals(notes),
            'note_count': note_count(notes),
            # 'note_length_transition_matrix': note_length_transition_matrix(notes)
        }
    else:
        pass
    return analysis

def analyze_midi_folder(folder_path, max_file_count=10000):
    results = {}

    current_file_count = 0
    # Walk recursively
    # for file_name in os.listdir(folder_path):
    #     if file_name.endswith('.mid') or file_name.endswith('.midi'):
    #         current_file_count += 1
    #         if current_file_count > max_file_count:
    #             break
    #         else:
    #             file_path = os.path.join(folder_path, file_name)
    #             print(f"Analyzing file {current_file_count} of {max_file_count} - {file_name}")
    #             results[file_name] = analyze_midi(file_path)
    global current_idx
    global indices_to_analyze

    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            # if current_idx not in indices_to_analyze:
            #     current_idx += 1
            #     continue
            # else:
            #     current_idx += 1
                
            if file_name.endswith('.mid') or file_name.endswith('.midi'):
                current_file_count += 1
                if current_file_count > max_file_count:
                    break
                else:
                    file_path = os.path.join(root, file_name)
                    print(f"Analyzing file {current_file_count} of {max_file_count} - {file_name}")
                    results[file_name] = analyze_midi(file_path)
    
    current_idx = 0
    
    return results

In [5]:
def average_analysis_results(analysis_results):
    average_results = {
        "pitch_counts": dict[int, float](),
        "pitch_class_distribution": dict[int, float](),
        "note_count": int(0)
    }
    
    n_files = len(analysis_results)

    for file_name, analysis in analysis_results.items():
        for key, value in analysis.items():
            if key == 'pitch_counts':
                for pitch, v in value.items():
                    if pitch not in average_results["pitch_counts"]:
                        average_results["pitch_counts"][pitch] = v / n_files
                    else:
                        average_results["pitch_counts"][pitch] += v / n_files
            elif key == 'pitch_class_distribution':
                for pitch_class, v in value.items():
                    if pitch_class not in average_results["pitch_class_distribution"]:
                        average_results["pitch_class_distribution"][pitch_class] = v / n_files
                    else:
                        average_results["pitch_class_distribution"][pitch_class] += v / n_files
            elif key == 'note_count':
                average_results[key] += value / n_files
            else:
                pass
    
    return average_results

# Hooktheory

In [6]:
folder_path = hooktheory_midi_dir
n_file_to_analyze = 29038

analysis_results = analyze_midi_folder(folder_path, n_file_to_analyze)

Analyzing file 1 of 29038 - d_deerhunter_back-to-the-middle_Solo_AQodPXyKoDl.mid
Analyzing file 2 of 29038 - t_the-beatles_a-hard-days-night_Intro_ZwxK_QNbxed.mid
Analyzing file 3 of 29038 - n_niall-horan_still_Bridge_jDgXdDVegKl.mid
Analyzing file 4 of 29038 - a_ashe_love-is-not-enough_Pre-Chorus_yvgPQdedxYq.mid
Analyzing file 5 of 29038 - b_bryan-scary_operaland_Chorus_DpgvRAkdgad.mid
Analyzing file 6 of 29038 - j_jimmy-fontanez_urban-lullaby_Verse_Wegl_wa-mrY.mid
Analyzing file 7 of 29038 - h_hirohiko-takayama_hudsons-adventure-island-iii---thunder-clash_Instrumental_yvmrLZGzxOW.mid
Analyzing file 8 of 29038 - h_hitomi-yaida_ashita-kara-no-tegami_Intro and Verse_AaoGbakPxeQ.mid
Analyzing file 9 of 29038 - b_billy-idol_white-wedding_Chorus_eWxLdzOpxaK.mid
Analyzing file 10 of 29038 - l_lawrence_the-heartburn-song_Verse_ROmNkROngNw.mid
Analyzing file 11 of 29038 - m_miguel_goingtohell_Verse_zngREQNMmJj.mid
Analyzing file 12 of 29038 - b_bensound_elevator-bossa-nova_Intro and Verse_d_g

In [7]:
n_file_to_print = 1

current_file = 0

# Print results
for file_name, analysis in analysis_results.items():
    if current_file < n_file_to_print:
        current_file += 1
        print(f"{current_file} of {n_file_to_analyze} result - {file_name}:")

        for key, value in analysis.items():
            print(f"  {key}: {value}")
    else:
        break

1 of 29038 result - d_deerhunter_back-to-the-middle_Solo_AQodPXyKoDl.mid:
  pitch_counts: {71: 14, 59: 2, 62: 2, 66: 16, 35: 2, 73: 6, 68: 24, 57: 2, 61: 12, 64: 14, 33: 2, 69: 2, 42: 2, 40: 2, 37: 2}
  pitch_class_distribution: {11: 18, 2: 2, 6: 18, 1: 20, 8: 24, 9: 6, 4: 16}
  note_count: 104


In [8]:
average_results = average_analysis_results(analysis_results)

# Print average results
print("Average results:")
for key, value in average_results.items():
    print(f"  {key}: {value}")

# Save average results to file
# with open('average_results.json', 'w') as f:
#     json.dump(average_results, f, indent=4)

# Save analysis results to file
# with open('analysis_results.json', 'w') as f:
#     json.dump(analysis_results, f, indent=4)

Average results:
  pitch_counts: {71: 3.872408568082833, 59: 4.522832150974202, 62: 6.751015910186446, 66: 5.19756870307855, 35: 1.1797300089537588, 73: 2.3271575177352286, 68: 3.8438597699562633, 57: 4.213926578965061, 61: 4.0318892485704625, 64: 6.722294923892708, 33: 1.4608788484054398, 69: 5.375060265858358, 42: 0.5163578758867827, 40: 0.8095598870445753, 37: 0.7086920586817417, 65: 5.235381224602055, 67: 6.013533989944063, 72: 3.1304153178590566, 38: 1.0582684757903464, 60: 5.295164956263996, 77: 1.1147461946414903, 75: 1.4570907087264502, 44: 0.23121427095530092, 56: 2.1156415731109797, 63: 4.265169777532585, 32: 0.7710586128521415, 58: 3.0627109305045597, 51: 0.740168055651224, 55: 3.3200289276116974, 27: 0.38518493009161037, 34: 1.0257937874509342, 36: 1.0498312555961187, 76: 1.838969626007215, 74: 2.720263103519285, 78: 0.969384943866664, 79: 0.8744059508230669, 81: 0.5952889317446202, 82: 0.34024381844480045, 70: 3.385908120393675, 45: 0.27822164060885973, 80: 0.4550244507197

In [9]:
# Horizontal bar chart for pitch counts using plotly express (with y is the pitch and x is the count)
fig = px.bar(
    x=list(average_results['pitch_counts'].values()), 
    y=list(average_results['pitch_counts'].keys()), 
    orientation='h'
)

fig.update_layout(
    title=f'Hooktheory (n = {n_file_to_analyze})', 
    xaxis_title='Số lần xuất hiện trung bình',
    yaxis_title='Cao độ'
)

# Show labels 0, 5, 10, 15, ... in y-axis
y_axis_tickvals = list(range(0, max(average_results['pitch_counts'].keys()) + 1, 5))
fig.update_yaxes(tickvals=y_axis_tickvals)

# Show labels 0, 1, 2, 3, ... in x-axis
x_axis_tickvals = list(range(
    0, 
    int(max(average_results['pitch_counts'].values())) + 1 + 1
))
fig.update_xaxes(tickvals=x_axis_tickvals)

# Change the color set to make it easier to see when printed in black and white
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)

# Change the background color to make it easier to see when printed in black and white
fig.update_layout(plot_bgcolor='white')

# Show the vertical grid lines
fig.update_layout(xaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgb(158,202,225)'))

# Make the colums overlay the grid lines
fig.update_layout(barmode='overlay')

# Make the font size bigger
fig.update_layout(font=dict(size=13))

# Make the plot ratio x:y
fig.update_layout(
    autosize=False,
    width=450,
    height=1200,
)

fig.show()


# Pitch class distribution
fig = px.bar(x=list(average_results['pitch_class_distribution'].keys()), y=list(average_results['pitch_class_distribution'].values()))
fig.update_layout(
    title=f'Hooktheory (n = {n_file_to_analyze})', 
    yaxis_title='Số lần xuất hiện trung bình',
    xaxis_title='Cao độ cơ bản'
)

# Show labels 0, 1, 2, 3, ... in x-axis
x_axis_tickvals = list(range(0, max(average_results['pitch_class_distribution'].keys()) + 1))
fig.update_xaxes(tickvals=x_axis_tickvals)

# Change the color set to make it easier to see when printed in black and white
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)

# Change the background color to make it easier to see when printed in black and white
fig.update_layout(plot_bgcolor='white')

# Show the horizontal grid lines
fig.update_layout(yaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgb(158,202,225)'))

# Make the font size bigger
fig.update_layout(font=dict(size=13))

# Make the plot ratio x:y
fig.update_layout(
    autosize=False,
    width=650,
    height=400,
)

fig.show()

# Note count
print(f"Số lượng nốt nhạc trung bình trong một đoạn nhạc từ Hooktheory (n = {n_file_to_analyze}): {average_results['note_count']}")

Số lượng nốt nhạc trung bình trong một đoạn nhạc từ Hooktheory (n = 29038): 117.50420139127878


# Model Output

In [10]:
folder_path = model_output_dir

# Count midi files in the folder path
n_file_to_analyze = 0
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.endswith('.mid') or file_name.endswith('.midi'):
            n_file_to_analyze += 1

analysis_results = analyze_midi_folder(folder_path, n_file_to_analyze)

Analyzing file 1 of 135 - 51_0.mid
Analyzing file 2 of 135 - 14_0.mid
Analyzing file 3 of 135 - 103_0.mid
Analyzing file 4 of 135 - 91_0.mid
Analyzing file 5 of 135 - 29_0.mid
Analyzing file 6 of 135 - 48_0.mid
Analyzing file 7 of 135 - 88_0.mid
Analyzing file 8 of 135 - 127_0.mid
Analyzing file 9 of 135 - 75_0.mid
Analyzing file 10 of 135 - 30_0.mid
Analyzing file 11 of 135 - 77_0.mid
Analyzing file 12 of 135 - 32_0.mid
Analyzing file 13 of 135 - 125_0.mid
Analyzing file 14 of 135 - 118_0.mid
Analyzing file 15 of 135 - 93_0.mid
Analyzing file 16 of 135 - 101_0.mid
Analyzing file 17 of 135 - 53_0.mid
Analyzing file 18 of 135 - 16_0.mid
Analyzing file 19 of 135 - 8_0.mid
Analyzing file 20 of 135 - 97_0.mid
Analyzing file 21 of 135 - 105_0.mid
Analyzing file 22 of 135 - 57_0.mid
Analyzing file 23 of 135 - 12_0.mid
Analyzing file 24 of 135 - 73_0.mid
Analyzing file 25 of 135 - 36_0.mid
Analyzing file 26 of 135 - 121_0.mid
Analyzing file 27 of 135 - 123_0.mid
Analyzing file 28 of 135 - 71_

In [11]:
n_file_to_print = 1

current_file = 0

# Print results
for file_name, analysis in analysis_results.items():
    if current_file < n_file_to_print:
        current_file += 1
        print(f"{current_file} of {n_file_to_analyze} result - {file_name}:")

        for key, value in analysis.items():
            print(f"  {key}: {value}")
    else:
        break

1 of 135 result - 51_0.mid:
  pitch_counts: {67: 15, 78: 1, 76: 6, 50: 6, 52: 12, 65: 3, 60: 14, 57: 6, 62: 7, 72: 3, 38: 1, 69: 9, 45: 3, 55: 5, 64: 8, 77: 3, 59: 7, 74: 5, 83: 2, 36: 1, 48: 4, 43: 7, 73: 1, 53: 3, 71: 1, 58: 2, 81: 2, 70: 1, 47: 1, 40: 2, 88: 6, 79: 1, 41: 1}
  pitch_class_distribution: {7: 28, 6: 1, 4: 34, 2: 19, 5: 10, 0: 22, 9: 20, 11: 11, 1: 1, 10: 3}
  note_count: 149


In [12]:
average_results = average_analysis_results(analysis_results)

# Print average results
print("Average results:")
for key, value in average_results.items():
    print(f"  {key}: {value}")

# Save average results to file
# with open('average_results.json', 'w') as f:
#     json.dump(average_results, f, indent=4)

# Save analysis results to file
# with open('analysis_results.json', 'w') as f:
#     json.dump(analysis_results, f, indent=4)

Average results:
  pitch_counts: {67: 11.629629629629632, 78: 0.8962962962962973, 76: 5.185185185185187, 50: 3.6074074074074063, 52: 8.940740740740742, 65: 4.096296296296296, 60: 11.237037037037036, 57: 4.311111111111111, 62: 6.940740740740744, 72: 4.688888888888888, 38: 0.1259259259259259, 69: 4.488888888888887, 45: 1.7999999999999983, 55: 5.874074074074074, 64: 6.955555555555557, 77: 3.6592592592592554, 59: 7.318518518518523, 74: 3.585185185185183, 83: 1.3185185185185178, 36: 2.155555555555554, 48: 5.940740740740743, 43: 7.770370370370374, 73: 0.7481481481481489, 53: 2.5703703703703686, 71: 1.9259259259259243, 58: 0.8074074074074084, 81: 0.9851851851851862, 70: 0.6740740740740748, 47: 1.607407407407406, 40: 1.6148148148148131, 88: 3.199999999999999, 79: 1.8962962962962946, 41: 2.955555555555553, 68: 0.2740740740740742, 56: 0.6518518518518523, 66: 0.85925925925926, 61: 0.20000000000000004, 75: 0.3925925925925929, 84: 0.5925925925925931, 63: 0.5481481481481486, 46: 0.22222222222222235,

In [13]:
# Horizontal bar chart for pitch counts using plotly express (with y is the pitch and x is the count)
fig = px.bar(
    x=list(average_results['pitch_counts'].values()), 
    y=list(average_results['pitch_counts'].keys()), 
    orientation='h'
)

fig.update_layout(
    title=f'GPT (n = {n_file_to_analyze})', 
    xaxis_title='Số lần xuất hiện trung bình',
    yaxis_title='Cao độ'
)

# Show labels 0, 5, 10, 15, ... in y-axis
y_axis_tickvals = list(range(0, max(average_results['pitch_counts'].keys()) + 1, 5))
fig.update_yaxes(tickvals=y_axis_tickvals)

# Show labels 0, 1, 2, 3, ... in x-axis
x_axis_tickvals = list(range(
    0, 
    int(max(average_results['pitch_counts'].values())) + 1 + 1
))
fig.update_xaxes(tickvals=x_axis_tickvals)

# Change the color set to make it easier to see when printed in black and white
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)

# Change the background color to make it easier to see when printed in black and white
fig.update_layout(plot_bgcolor='white')

# Show the vertical grid lines
fig.update_layout(xaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgb(158,202,225)'))

# Make the colums overlay the grid lines
fig.update_layout(barmode='overlay')

# Make the font size bigger
fig.update_layout(font=dict(size=13))

# Make the plot ratio x:y
fig.update_layout(
    autosize=False,
    width=450,
    height=1200,
)

fig.show()


# Pitch class distribution
fig = px.bar(x=list(average_results['pitch_class_distribution'].keys()), y=list(average_results['pitch_class_distribution'].values()))
fig.update_layout(
    title=f'GPT (n = {n_file_to_analyze})', 
    yaxis_title='Số lần xuất hiện trung bình',
    xaxis_title='Cao độ cơ bản'
)

# Show labels 0, 1, 2, 3, ... in x-axis
x_axis_tickvals = list(range(0, max(average_results['pitch_class_distribution'].keys()) + 1))
fig.update_xaxes(tickvals=x_axis_tickvals)

# Change the color set to make it easier to see when printed in black and white
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)

# Change the background color to make it easier to see when printed in black and white
fig.update_layout(plot_bgcolor='white')

# Show the horizontal grid lines
fig.update_layout(yaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgb(158,202,225)'))

# Make the font size bigger
fig.update_layout(font=dict(size=13))

# Make the plot ratio x:y
fig.update_layout(
    autosize=False,
    width=650,
    height=400,
)

fig.show()

# Note count
print(f"Số lượng nốt nhạc trung bình trong một đoạn nhạc từ GPT (n = {n_file_to_analyze}): {average_results['note_count']}")

Số lượng nốt nhạc trung bình trong một đoạn nhạc từ GPT (n = 135): 136.6666666666667
